Import required packages

In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy import distance
import folium

Import Foursquare credentials from json file created to preserve secrecy and store in global variables CLIENT_ID and CLIENT_SECRET

In [2]:
import json
globals().update(json.loads(open("foursquareCredentials.json", "r") .read()))
VERSION = '20181124' # Foursquare API version

Get longitude and latitude of location

In [3]:
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode("Rumbach Sebestyén u., Budapest, 1075 Hungary")
lat = location.latitude
long = location.longitude

Define function degreeLen which for a given lat and long returns the length of a degree of lat and long in km

In [4]:
def degreeLen(lat, long):
    """Returns the lengths of a single degree of latitude and longitude in km for a given latitude and longitude
    """
    # get length of 1 degree at current latidude and logitude
    latDegree = distance.distance((lat - 0.5, long), (lat + 0.5, long)).km
    longDegree = distance.distance((lat, long - 0.5), (lat, long + 0.5)).km
    return latDegree, longDegree
degreeLen(lat, long)

(111.18052869040775, 75.34729533841742)

In [5]:
def grid(lat, long, radius, tick):
    """Returns a DataFrame whose rows are a grid of points within a circle around a location
    :param lat: latitude of centre of grid
    :param long: longitude of centre of grid
    :param radius: radius of grid in km
    :param tick: distance between points in grid in km
    :returns: pandas DataFrame with a row for each point in the grid and 
              columns:
                  'Name': string of latitude and longitude of point separated by a comma
                  'Latitude': latitude of point
                  'Longitude': longitude of point
    """
    # get length of 1 degree at current latidude and logitude
    latDegree, longDegree = degreeLen(lat, long)
    # get length of 1 tick in degrees
    latTick = tick / latDegree
    longTick = tick / longDegree
    # get coordinates of point which is one radius to the South and West of (lat, long)
    bottomLeft = (lat - radius / latDegree, long - radius / longDegree)
    # get tick points along axis
    gridLats = np.arange(2 * radius / tick + 1) * latTick + bottomLeft[0]
    gridLongs = np.arange(2 * radius / tick + 1) * longTick + bottomLeft[1]
    grid = []
    for lat in gridLats:
        for long in gridLongs:
            if distance.distance((location.latitude, location.longitude), (lat, long)).km <= radius:
                grid.append([str(lat) + ', ' + str(long), lat, long])
    grid = pd.DataFrame(grid)
    grid.columns = ['Name', 'Latitude', 'Longitude']
    return grid

Check using Folium

In [8]:
g = grid(lat, long, 2, 0.2)
# create map
map_grid = folium.Map(location=[lat, long], zoom_start=13)

for i in g.index:
    label = folium.Popup(g.at[i, 'Name'], parse_html=True)
    folium.CircleMarker([g.at[i, 'Latitude'], g.at[i, 'Longitude']],
                        radius=2,
                        popup=label,
                        fill=True,
                        fill_opacity=0.5).add_to(map_grid)
       
map_grid

In [7]:
g

,Name,Latitude,Longitude
0,"47.47904203862642, 19.0598899",47.479042,19.059890
1,"47.480840914763775, 19.04927239878238",47.480841,19.049272
2,"47.480840914763775, 19.051926774086787",47.480841,19.051927
3,"47.480840914763775, 19.054581149391193",47.480841,19.054581
4,"47.480840914763775, 19.057235524695596",47.480841,19.057236
5,"47.480840914763775, 19.0598899",47.480841,19.059890
6,"47.480840914763775, 19.062544275304408",47.480841,19.062544
7,"47.480840914763775, 19.065198650608814",47.480841,19.065199
8,"47.480840914763775, 19.067853025913216",47.480841,19.067853
9,"47.480840914763775, 19.070507401217622",47.480841,19.070507
